In [17]:
import os
os.environ["NUMEXPR_MAX_THREADS"] = "32"
os.environ["JULIA_NUM_THREADS"] = "32"
import tushare as ts
# pro = ts.pro_api('20241114201856-4f34155e-3cd2-4a42-b798-df1713e24f07')
pro = ts.pro_api('20240522230128-05e173d4-a1b9-4220-a737-b3fc07a02766',200)
# pro = ts.pro_api('20240522230128-9a49d070-5302-4d5b-a88e-a523e8d84f94')
pro._DataApi__http_url = 'http://tsapi.majors.ltd:7000'


In [18]:
# # coding=utf-8
# from xtquant import xtdata
# # 获取市场所有指数id
# ret_sector_list = xtdata.get_sector_list()
# print(f'获取板块目录: {ret_sector_list}')

# xt_sector_index_list = xtdata.get_stock_list_in_sector("深市指数")
# print(xt_sector_index_list)
# # 获取迅投板块指数合约信息
# for item in xt_sector_index_list:
#     xt_sector_index_info = xtdata.get_instrument_detail(item)
#     # xt_sector_index = xt_sector_index_list[0]
#     print(f"id:{item} :{xt_sector_index_info['InstrumentName']}")
    
# xt_sector_index_list = xtdata.get_stock_list_in_sector("沪市指数")
# print(xt_sector_index_list)
# # 获取迅投板块指数合约信息
# for item in xt_sector_index_list:
#     xt_sector_index_info = xtdata.get_instrument_detail(item)
#     # xt_sector_index = xt_sector_index_list[0]
#     print(f"id:{item} :{xt_sector_index_info['InstrumentName']}")



In [19]:
from pandas import DataFrame
import pandas as pd
def coverBatchStockToDataFrame(existing_data:DataFrame):
    data={}
    for item in existing_data["close"].T.columns:
        result = pd.concat([
            existing_data["open"].loc[item], 
            existing_data["high"].loc[item],
            existing_data["low"].loc[item],
            existing_data["close"].loc[item],
            existing_data["volume"].loc[item],
            existing_data["amount"].loc[item],
            existing_data["preClose"].loc[item]], axis=1,keys=['open', 'high', 'low', 'close', 'volume', 'amount', 'preClose'])
        result["openinterest"]=0
        result.set_index(keys=result.index,inplace=True)
        
        data[item]=result
    return data

In [20]:
# 数据时间
start_time="20211230"
end_time="20240510"
# 回测时间
trader_start_time="20220101"
trader_end_time="20240501"
from xtquant import xtdata
stocks = xtdata.get_stock_list_in_sector('沪深A股') 
stocks=[item for item in stocks if not item.startswith("688")]


In [21]:
# # 下载数据
# xtdata.download_history_data2(stock_list=["399303.SZ"], period='1d', start_time=start_time,end_time=end_time, incrementally=True,callback=lambda data:print(data))
# xtdata.download_history_data2(stock_list=stocks, period='1d', start_time=start_time,end_time=end_time, incrementally=True,callback=lambda data:print(data))


In [22]:
# 下载交易日数据,可缓存
import akshare as ak
import pandas as pd
trade_date_list = ak.tool_trade_date_hist_sina()
trade_date_list = trade_date_list.rename(columns={'trade_date': 'datetime'})
trade_date_list.set_index(pd.to_datetime(trade_date_list["datetime"]),inplace=True)
trade_date_list.drop(columns=['datetime'], inplace=True)
trade_date_list=trade_date_list[start_time:end_time]
# 行情数据
existing_data = xtdata.get_market_data_ex(
                    field_list=["time","open", "high", "low", "close", "volume", "amount", "preClose","suspendFlag"],stock_list=stocks,
                    period="1d",start_time=start_time,end_time=end_time,fill_data=False,dividend_type="back"
                )
# 基准数据
benchmark_data=xtdata.get_market_data_ex(
                    field_list=["time","open", "high", "low", "close", "volume", "amount", "preClose","suspendFlag"],stock_list=["399303.SZ"],
                    period="1d",start_time=start_time,end_time=end_time,fill_data=False,dividend_type="back"
                )["399303.SZ"]
benchmark_data.set_index(pd.to_datetime(benchmark_data.index),inplace=True)


In [ ]:
import os
import pandas as pd

def init_stk_factor(trade_date_list, file_path='stk_factor_data.csv'):
    print(f"开始缓存数据{trade_date_list.index.tolist()}")
    # 检查CSV文件是否存在
    if os.path.exists(file_path):
        # 如果文件存在，读取现有的 CSV 文件
        stk_factor_data = pd.read_csv(file_path)
    else:
        # 如果文件不存在，初始化一个空的 DataFrame
        stk_factor_data = pd.DataFrame()
    # 遍历每个交易日
    for trade_date in trade_date_list.index.tolist():
        try:
            # 检查当前的 trade_date 是否已经在现有数据中
            if stk_factor_data.empty or int(trade_date.strftime('%Y%m%d')) not in stk_factor_data['trade_date'].unique().tolist():
                print(f"Downloading data for trade_date: {trade_date}")
                # 如果不存在，下载新的数据
                new_data = pro.stk_factor_pro(trade_date=trade_date.strftime('%Y%m%d'))
                if os.path.exists(file_path):
                    new_data.to_csv(file_path, mode='a', header=False, index=False)
                else:
                    new_data.to_csv(file_path, mode='w', header=True, index=False)
            else:
                print(f"Data for trade_date {trade_date} already exists.")
        except:
            continue
    stk_factor_data = pd.read_csv(file_path)
    return stk_factor_data


In [ ]:
# 下载交易日的指标数据
stk_factor_data=init_stk_factor(trade_date_list,"stk_factor_data.csv")

开始缓存数据[Timestamp('2021-12-30 00:00:00'), Timestamp('2021-12-31 00:00:00'), Timestamp('2022-01-04 00:00:00'), Timestamp('2022-01-05 00:00:00'), Timestamp('2022-01-06 00:00:00'), Timestamp('2022-01-07 00:00:00'), Timestamp('2022-01-10 00:00:00'), Timestamp('2022-01-11 00:00:00'), Timestamp('2022-01-12 00:00:00'), Timestamp('2022-01-13 00:00:00'), Timestamp('2022-01-14 00:00:00'), Timestamp('2022-01-17 00:00:00'), Timestamp('2022-01-18 00:00:00'), Timestamp('2022-01-19 00:00:00'), Timestamp('2022-01-20 00:00:00'), Timestamp('2022-01-21 00:00:00'), Timestamp('2022-01-24 00:00:00'), Timestamp('2022-01-25 00:00:00'), Timestamp('2022-01-26 00:00:00'), Timestamp('2022-01-27 00:00:00'), Timestamp('2022-01-28 00:00:00'), Timestamp('2022-02-07 00:00:00'), Timestamp('2022-02-08 00:00:00'), Timestamp('2022-02-09 00:00:00'), Timestamp('2022-02-10 00:00:00'), Timestamp('2022-02-11 00:00:00'), Timestamp('2022-02-14 00:00:00'), Timestamp('2022-02-15 00:00:00'), Timestamp('2022-02-16 00:00:00'), Timesta

In [ ]:
stk_factor_data = pd.read_csv("stk_factor_data.csv")
for item in stk_factor_data['trade_date'].unique().tolist():
    print(type(item))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [ ]:
# 下载涨跌停数据
# from pandas.core.frame import DataFrame
# df = pro.kpl_list(start_date=start_time, end_date=end_time,tag='跌停')
# df.to_csv(f"{start_time}~{end_time}跌停池.csv")
# df = pro.kpl_list(start_date=start_time, end_date=end_time,tag='涨停')
# df.to_csv(f"{start_time}~{end_time}涨停池.csv")
dieting=pd.read_csv(f"{start_time}~{end_time}跌停池.csv")
zhangting=pd.read_csv(f"{start_time}~{end_time}涨停池.csv")
zt = zhangting[zhangting['trade_date'] == int("20240510")]["ts_code"].tolist()
dt = dieting[dieting['trade_date'] == int("20240510")]["ts_code"].tolist()


In [ ]:
import datetime as datetime
import tushare as ts
# 获取st数据
def get_st(trade_date,df):
    result = df[(df.start_date<=trade_date) & ((df.end_date>=trade_date) | (df.end_date.isna()))]
    return result
def init_st(start_time,end_time):
    # 获取三年前日期，因为ST最多3年
    start_date = (datetime.datetime.strptime(start_time,'%Y%m%d') - datetime.timedelta(days=365*3)).strftime('%Y%m%d')
    # 获取三年内所有更名记录
    df = pro.namechange(start_date=start_date,end_date=end_time, fields='ts_code,name,start_date,end_date,ann_date,change_reason')
    # 筛选更名原因为ST或*ST的记录，去掉结束日期重复为None的
    df = df[(df['change_reason']=='ST') | (df['change_reason']=='*ST')] \
    .sort_values(by='end_date',ascending=True).drop_duplicates(subset=['ts_code','start_date'],keep='first')
    # 筛选目标日期在ST时间段的记录
    return df
# st_result=init_st(start_time,end_time)
# st_result.to_csv(f"{start_time}~{end_time}st池子.csv")
st_result=pd.read_csv(f"{start_time}~{end_time}st池子.csv")

In [ ]:
import backtrader as bt
import datetime
import pandas as pd

# class PandasData_more(bt.feeds.PandasData):
#     lines = ('ROE', 'EP', )  # 要添加的线
#     # 设置 line 在数据源上的列位置
#     params = dict(
#         ROE=-1,  # 设置新增指标的位置，-1表示自动按列明匹配数据
#         EP=-1
#         )
import backtrader as bt
class CustomPandasData(bt.feeds.PandasData):
    lines=("amount","suspendFlag")
    params=(
        ("datatime",None),
        ("open","open"),
        ("close","close"),
        ("low","low"),
        ("high","high"),
        ("volume","volume"),
        ("amount","amount"),
        ("openinterest",-1),
        ("suspendFlag","suspendFlag"),
        )

class StockSelectStrategy(bt.Strategy):
    params = dict(
        selnum=30,  # 设置持仓股数在总的股票池中的占比，如买入表现最好的前30只股票
        rperiod=1,  # 计算收益率的周期
        vperiod=6,  # 计算波动率的周期，过去6个月的波动率
        mperiod=2,  # 计算动量的周期，如过去2个月的收益
        reserve=0.05  # 5% 为了避免出现资金不足的情况，每次调仓都预留 5% 的资金不用于交易
    )
    def log(self, arg):
        print("进入log")
        print('{} {}'.format(self.datetime.date(), arg))

    def __init__(self):
        print("进入init")
        # 计算持仓权重，等权
        # self.perctarget = (1.0 - self.p.reserve) / self.p.selnum
        # # 循环计算每只股票的收益波动率因子
        # self.rs = {d:bt.ind.PctChange(d, period=self.p.rperiod) for d in self.datas}
        # self.vs = {d:1/(bt.ind.StdDev(ret, period=self.p.vperiod)+0.000001) for d,ret in self.rs.items()}
        # # 循环计算每只股票的动量因子
        # self.ms = {d:bt.ind.ROC(d, period=self.p.mperiod) for d in self.datas}
        # # 将 ep 和 roe 因子进行匹配
        # self.EP = {d:d.lines.EP for d in self.datas}
        # self.ROE = {d:d.lines.ROE for d in self.datas}
        # self.all_factors = [self.rs, self.vs, self.ms, self.EP, self.ROE]
    def getGuxi(trader_time,stock_code):
        # trader_time="20241204"
        start_time=(pd.to_datetime(trader_time)-datetime.timedelta(days=365)).strftime('%Y%m%d')
        data = xtdata.get_divid_factors(stock_code=stock_code, start_time='')
        if data is not None and not data.empty:
            target_data = data[(data.index >=start_time) & (data.index <= trader_time)]
            return target_data["interest"].sum()
    def get_unUpMarker(self,currentTime,datas):
        # 未上市的,
        stock_list=[]
        for item in datas:
            stock_info=xtdata.get_instrument_detail(stock_code=item._name)
            # print(f"name:{item._name} opendate{stock_info['OpenDate']}")
            stock_open_date=pd.to_datetime(stock_info["OpenDate"])
            delta=pd.to_datetime(currentTime)-stock_open_date
            if((not stock_open_date==pd.to_datetime("19700101")) and stock_open_date<pd.to_datetime(currentTime) and delta>datetime.timedelta(days=365)):
                # print(f"上市的 name:{item._name} opendate{stock_info['OpenDate']}")
                pass
            else:
                # print(f"未上市的 name:{item._name} opendate{stock_info['OpenDate']}")
                stock_list.append(item._name)
                
        return stock_list
    def get_ZDT(self,currentTime):
        stock_list=[]
        zt = zhangting[zhangting['trade_date'] == int(currentTime)]["ts_code"].tolist()
        dt = dieting[dieting['trade_date'] == int(currentTime)]["ts_code"].tolist()
        return zt,dt
    #stock_code = 'xxxxxx.SH'
    #target_date = '20200830'
    #若为ST，返回True；否则返回False
    def next(self):
        # 去除st的,,去除停牌
        print(f"进入next{self.data.datetime.date()}")
        unmarker=self.get_unUpMarker(currentTime=self.data.datetime.date(),datas=self.datas)
        # # 去除不在持仓内的昨日涨跌停的
        zt,dt=self.get_ZDT(self.data.datetime.date(-1).strftime('%Y%m%d'))
        st_stock = get_st(int(self.data.datetime.date(0).strftime('%Y%m%d')),st_result)
        print(f"st集合:{list(st_stock['ts_code'])}")
        print(f"未上市集合:{unmarker}")
        print(f"跌停:{dt}")
        print(f"涨停:{zt}")
        # self.stock_list=[]
        # for stock in self.datas:
        #     if((stock._name not in unmarker) and (stock._name not in zdt) and (stock._name not in yound) and (stock._name not in tingpai) and (stock._name not in st_stock)):
        #         self.stock_list.append(stock)
        # self.stock_list=get_dividend_ratio_filter_list(self.stock_list)
        # self.stock_list=get_dividend_ratio_filter_list(self.stock_list)
        hold_bond_name= [_p._name for _p in self.broker.positions if self.broker.getposition(_p).size > 0]
        print(f"当前持仓{hold_bond_name}")
        # guxiData=pd.DataFrame()
        filterData=[]
        for data in self.datas:
            if(data._name in unmarker or data._name in list(st_stock["ts_code"])):
                print(f"过滤未上市和st:{data._name}")
            elif(data.volume[0]==0 or data.suspendFlag[0]==1):
                print(f"过滤停牌{data._name}")
            elif(data._name not in hold_bond_name and data._name in zt and data._name in dt ):
                print(f"过滤不在持仓中的涨跌停{data._name}")
            else:
                # 
                guxi = self.getGuxi(self.data.datetime.date(-1).strftime('%Y%m%d'), data._name)
                # 如果股息不为 0，更新 DataFrame
                # if guxi > 0:
                # fuzhai = self.getFuzhai(self.data.datetime.date(-1).strftime('%Y%m%d'), data._name)
                filterData.append({"ts_code": [data._name], "guxi": [guxi]})
                print(filterData)
                # if(not self.position and ("600900.SH"==data._name or "600029.SH"==data._name )):
                #     cash = self.broker.get_cash()  # 当前账户的现金余额
                #     price = self.data.close[0]  # 当前的市场价格
                #     # 计算可购买的股数
                #     full_size = int(cash / price)
                #     # 计算半仓买入的股数
                #     half_size = full_size // 3  # 取一半股数
                #     # 执行买入操作
                #     self.buy(size=half_size)  # 买入半仓股数
                stock_name = data._name  # 获取股票名称
                kline_date = data.datetime.date(0)  # 获取当前K线日期
                open_price = data.open[0]  # 获取开盘价
                close_price = data.close[0]  # 获取收盘价
                volume = data.volume[0]  # 获取成交量
                amount = data.amount[0]  # 获取成交量
                suspendFlag=data.suspendFlag[0] 
                # 打印当前K线的信息
                print(f"昨日时间:{data.datetime.date(-1)}")
                print(f"日期: {kline_date}, 股票: {stock_name}, 开盘价: {open_price}, 收盘价: {close_price}, 成交量: {volume},成交额:{amount},是否停牌:{suspendFlag}")

# 实例化 cerebro
cerebro = bt.Cerebro()
# 按股票代码，依次循环传入数据
existing_data_list = list(existing_data.items())[0:100]
for index,stock in enumerate(dict(existing_data_list)):
    # 日期对齐
    existing_data[stock].set_index(pd.to_datetime(existing_data[stock].index),inplace=True)
    existing_data[stock]["openinterest"]=0
    # print(f"{index}:{len(existing_data)}")
    data_ = pd.merge(left=trade_date_list, right=existing_data[stock], left_index=True, right_index=True, how='left')
    # 缺失值处理：日期对齐时会使得有些交易日的数据为空，所以需要对缺失数据进行填充
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(0.0000001)
    # 导入数据
    datafeed = CustomPandasData(dataname=data_, 
                               fromdate=pd.to_datetime(trader_start_time),
                               todate=pd.to_datetime(trader_end_time),
                               timeframe=bt.TimeFrame.Days) # 将数据的时间周期设置为月度
    cerebro.adddata(datafeed, name=stock) # 通过 name 实现数据集与股票的一一对应
    # print(f"{stock} Done !") 
# 初始资金 100,000,000    
cerebro.broker.setcash(100000000.0) 
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003) 
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001) 
# 将编写的策略添加给大脑，别忘了 ！
cerebro.addstrategy(StockSelectStrategy)
# 返回收益率时序
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')
# 添加观察者 PyFolio
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')

#########  注意  #########
# PyFolio 分析器返回的收益也是月度收益，但是绘制的各种收益分析图形会有问题，有些图绘制不出来
result = cerebro.run()
# 得到收益率时序
ret = pd.Series(result[0].analyzers._TimeReturn.get_analysis())


In [ ]:
%matplotlib inline
import pyfolio as pf
import quantstats as qs
import matplotlib.pyplot as plt  # 由于 Backtrader 的问题，此处要求 pip install matplotlib==3.2.2
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['font.sans-serif'] = ['DengXian']
plt.rcParams['axes.unicode_minus'] = False  
portfolio_stats = result[0].analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
print(returns)
pf.create_full_tear_sheet(returns)
qs.reports.full(returns)

qs.stats.sharpe(returns)
qs.stats.conditional_value_at_risk(returns)

help(qs.stats.conditional_value_at_risk)
qs.extend_pandas()
# 基准收益序列
benchmark_pnl=benchmark_data[trader_start_time:trader_end_time]
# 计算每日收益率
benchmark_pnl1 = benchmark_pnl["close"].pct_change()  # pct_change() 会计算前一天的相对收益率
# 计算累计净值
benchmark_cumulative = (1 + benchmark_pnl1).cumprod()  # 累计收益曲线
benchmark_cumulative = benchmark_cumulative.fillna(1)  # 填充 NaN 值，通常是第一个数据点
benchmark_cumulative.sharpe()
returns.index=returns.index.tz_convert(None)
qs.plots.snapshot(benchmark_cumulative, title='中证2000',fontname="DengXian")
qs.reports.html(returns, benchmark=benchmark_cumulative,benchmark_title="中证2000", output='stats.html', title='Report')  # 会在本地目录生成 html 文件，用游览器打开即可


In [ ]:
# import matplotlib.font_manager as fm

# # 获取所有字体
# font_paths = fm.findSystemFonts(fontpaths=None, fontext='ttf')

# # 打印所有字体路径
# for font_path in font_paths:
#     print(font_path)

# # 也可以查看字体的名字
# font_names = [fm.FontProperties(fname=font).get_name() for font in font_paths]
# for font_name in font_names:
#     print(font_name)


In [ ]:
# 打印结果
# 提取收益序列
pnl = pd.Series(result[0].analyzers._TimeReturn.get_analysis())
# 计算累计收益
cumulative = (pnl + 1).cumprod()
# 计算回撤序列
max_return = cumulative.cummax()
drawdown = (cumulative - max_return) / max_return

# 计算收益评价指标
import pyfolio as pf
# 按年统计收益指标
perf_stats_year = (pnl).groupby(pnl.index.to_period('y')).apply(lambda data: pf.timeseries.perf_stats(data)).unstack()
# 统计所有时间段的收益指标
perf_stats_all = pf.timeseries.perf_stats((pnl)).to_frame(name='all')
perf_stats = pd.concat([perf_stats_year, perf_stats_all.T], axis=0)
perf_stats_ = round(perf_stats,4).reset_index()


# 绘制图形
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  # 导入设置坐标轴的模块
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['font.sans-serif'] = ['DengXian']
plt.rcParams['axes.unicode_minus'] = False  


fig, (ax0, ax1) = plt.subplots(2,1, gridspec_kw = {'height_ratios':[1.5, 4]}, figsize=(20,8))
cols_names = ['日期', '年化\n收益', '累计\n收益', '年化\n波动率',
              '夏普\n比率', '卡尔玛\n比率', '稳定性', '最大\n回撤',
              '欧米茄\n比率', '索提诺\n比率', '偏度', '峰度', '尾部\n比率',
              '日内风险\n价值']

# 绘制表格
ax0.set_axis_off()  # 除去坐标轴
table = ax0.table(cellText = perf_stats_.values, 
                bbox=(0,0,1,1),  # 设置表格位置， (x0, y0, width, height)
                rowLoc = 'right',  # 行标题居中
                cellLoc='right' ,
                colLabels = cols_names, # 设置列标题
                colLoc = 'right',  # 列标题居中
                edges = 'open' # 不显示表格边框
                )
table.set_fontsize(15)
# 绘制累计收益曲线
ax2 = ax1.twinx()
ax3 = ax1.twinx()
ax1.yaxis.set_ticks_position('right') # 将回撤曲线的 y 轴移至右侧
ax2.yaxis.set_ticks_position('left') # 将累计收益曲线的 y 轴移至左侧
ax3.yaxis.set_ticks_position('left') # 将累计收益曲线的 y 轴移至左侧
# 绘制回撤曲线
ax1.set_ylim(-1, 0)  # 假设回撤最大为 -100%，最小为 0
drawdown.plot.area(ax=ax1, label='回撤曲线', rot=0, alpha=0.3, fontsize=13, grid=False)
min_value = min(cumulative.min(), benchmark_cumulative.min())
max_value = max(cumulative.max(), benchmark_cumulative.max())
# 设置相同的 y 轴范围
ax2.set_ylim(min_value, max_value)
ax3.set_ylim(min_value, max_value)
# 绘制累计收益曲线
(cumulative).plot(ax=ax2, color='#F1C40F' , lw=3.0, label='累计收益', rot=0, fontsize=13, grid=False)
# 绘制基准收益曲线
(benchmark_cumulative).plot(ax=ax3, color='#FF040F' , lw=3.0, label='基准收益', rot=0, fontsize=13, grid=False)
# 不然 x 轴留有空白
ax2.set_xbound(lower=cumulative.index.min(), upper=cumulative.index.max())
# 主轴定位器：每 5 个月显示一个日期：根据具体天数来做排版
ax2.xaxis.set_major_locator(ticker.MultipleLocator(100)) 
# 同时绘制双轴的图例
h1,l1 = ax1.get_legend_handles_labels()
h2,l2 = ax2.get_legend_handles_labels()
h3,l3 = ax3.get_legend_handles_labels()
plt.legend(h1+h2+h3,l1+l2+l3,fontsize=12, loc='upper left', ncol=1)

fig.tight_layout() # 规整排版
plt.show()

In [ ]:
# df = pro.daily_basic(start_date=start_time,end_time=end_time)
# df

In [ ]:
# df[df.ts_code=="002303.SZ"]

In [ ]:
# df.sort_values(by="dv_ttm",ascending=False)

In [ ]:
# for index, row in df.iterrows():
#     ts_code = row['ts_code']
#     dv_ttm = row['dv_ttm']
#     print(f"ts_code: {ts_code}, dv_ttm: {dv_ttm}")
#     stock_info=xtdata.get_instrument_detail(stock_code=ts_code)
#     print(stock_info)

In [ ]:
# stocks = xtdata.get_stock_list_in_sector('沪深A股') 
# stocks=[item for item in stocks if not item.startswith("688")]
# xtdata.download_financial_data2(stocks,table_list=['Balance','Income','CashFlow','Capital','Top10FlowHolder','Top10Holder','HolderNum','PershareIndex'],callback=lambda data:print(data))


In [ ]:
# xtdata.download_financial_data2(stock_list=['601933.SH','301388.SZ'],table_list=['PershareIndex'],start_time="20180427",end_time="20200427",callback=lambda data:print(data))
# # xtdata.download_financial_data(stock_list=['600519.SH','301388.SZ'],table_list=['Balance'],start_time="20180427",end_time="20200427",incrementally=False)


In [ ]:
# # xtdata.download_financial_data2(['600660.SH'],table_list=['Balance','Income'],callback=lambda data:print(data))
# xtdata.download_financial_data(stock_list=['600660.SH'],table_list=['Balance'],start_time="20180427",end_time="20200427",incrementally=False)


In [ ]:
# from xtquant import xtdata
# import pandas as pd
# table_list = ['Balance']
# res = xtdata.get_financial_data(stock_list=['601933.SH'])
# print(res['601933.SH'].keys())
# print(res['601933.SH']["PershareIndex"])

In [ ]:
# import akshare as ak
# pd.options.display.float_format = '{:,.0f}'.format
# # pd.options.display.float_format = None

# data=ak.stock_zcfz_em("20240331").round(2)
# data

In [ ]:
import tushare as ts
import pandas as pd
import numpy as np

def HSL(security_list, check_date, N=5, unit='1d', include_now=True):
    '''
    计算换手线（HSL）及其N日移动平均（MAHSL）
    security_list: 股票代码列表
    check_date: 查询日期
    N: N日简单移动平均的天数
    unit: 数据的时间单位
    include_now: 是否包括当天的成交数据
    '''
    # 存储最终结果
    result = {}
    
    # 获取相关股票的成交量和流通股本数据
    for stock in security_list:
        # 获取股票的日线数据，包含成交量（vol）和流通股本（circ_market_cap）
        df = pro.daily(ts_code=stock, start_date=check_date, end_date=check_date)
        print(df)
        if df.empty:
            result[stock] = {'HSL': None, 'MAHSL': None}
            continue
        
        # 提取成交量和流通股本
        volume = df.iloc[0]['vol']  # 成交量（手）
        circ_market_cap = df.iloc[0]['circ_mv']  # 流通股本（元）
        
        # 假设SETCODE == 0 或 1表示市场类型的条件
        if stock.startswith('00') or stock.startswith('60'):  # 沪深市场
            HSL_value = 100 * volume  # 按成交量计算换手线
        else:
            HSL_value = volume / (circ_market_cap / 100)  # 按流通股本计算换手线
        
        # 获取过去N天的HSL数据
        past_data = pro.daily(ts_code=stock, start_date='20210101', end_date=check_date)  # 假设2021年1月1日为查询起点
        past_data['HSL'] = np.where(
            past_data['ts_code'].str.startswith('00') | past_data['ts_code'].str.startswith('60'),
            100 * past_data['vol'],
            past_data['vol'] / (past_data['circ_mv'] / 100)
        )
        
        # 计算过去N日的MAHSL
        past_data['MAHSL'] = past_data['HSL'].rolling(window=N).mean()
        
        # 获取最近的MAHSL
        if len(past_data) >= N:
            latest_MAHSL = past_data['MAHSL'].iloc[-1]
        else:
            latest_MAHSL = None
        
        # 存储结果
        result[stock] = {
            'HSL': HSL_value,
            'MAHSL': latest_MAHSL
        }
    
    return result

# 示例调用
security_list = ['000001.SZ', '600000.SH']  # 你要查询的股票代码列表
check_date = '2024-12-05'  # 要查询的日期
N = 5  # N日简单移动平均

result = HSL(security_list, check_date, N)
print(result)
